<a href="https://colab.research.google.com/github/Edisuism/Machine_Learning/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2


In this page, I will develop a machine learning algorithm from scratch and then test it on a sample dataset

Steps involved:
- Find sample dataset
- Choose learner algorithm and note down input/output
- Preprocess sample dataset to input requirements and split into test/training sets 
- code learner algorithm **(only this needs to be coded, preprocessing can use from libaries like SKlearn)**
- train algorithm and then apply to test set

# Introduction

new text


# Exploration

# Methodology

Import data


In [0]:
import numpy as np
import pandas as pd
from numpy import log2 as log
import random

url = 'https://raw.githubusercontent.com/Edisuism/Machine_Learning/master/play_tennis.csv'
df = pd.read_csv(url)
print (len(df))
df.head()

Split into training and testing sets

In [0]:
def train_test_split(df, test_size):
  test_size = round(test_size * len(df))
  indices = df.index.tolist()
  test_indices = random.sample(population = indices, k = test_size)
  test_df = df.loc[test_indices]
  train_df = df.drop(test_indices)
  
  return train_df, test_df



In [0]:
train_df, test_df = train_test_split(df, test_size = 0.3)

In [0]:
print (len(train_df))
train_df.head()

In [0]:
print (len(test_df))
test_df.head()

Find Entropy of Class


In [0]:
entropy_class = 0  
values = train_df.play.unique()  
for value in values:
    fraction = df.play.value_counts()[value]/len(df.play)  
    entropy_class += -fraction*np.log2(fraction)
    
print (entropy_class)

Find Entropy and Information Gain of Attributes

In [0]:
def attribute_E_IG(target_attribute):
  attribute = target_attribute
  eps = np.finfo(float).eps #In case of 0 denominator
  target_variables = train_df.play.unique() 
  variables = train_df[attribute].unique()    
  entropy_attribute = 0
  for variable in variables:
      entropy_each_feature = 0
      for target_variable in target_variables:
          num = len(train_df[attribute][train_df[attribute]==variable][train_df.play ==target_variable]) 
          den = len(train_df[attribute][train_df[attribute]==variable])  
          fraction = num/(den+eps)  
          entropy_each_feature += -fraction*log(fraction+eps) #entropy for one feature 
      fraction2 = den/len(train_df)
      entropy_attribute += -fraction2*entropy_each_feature   #all the entropy for attribute
  E_final = abs(entropy_attribute)
  IG_final = entropy_class - E_final
  return E_final, IG_final, target_attribute

print (attribute_E_IG('outlook'))
print (attribute_E_IG('temp'))
print (attribute_E_IG('humidity'))
print (attribute_E_IG('wind'))

Notes:
Gini index = how much uncertainty there is in a node
,Information gain = how much uncertainty is removed in a node

# Evaluation

# Conclusion

# Ethical